In [2]:
# core Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import sys
import io
warnings.filterwarnings('ignore')

# fix windows console encoding
#if sys.platform == 'win32':
 #   sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8', errors='replace')
if sys.platform == 'win32' and hasattr(sys.stdout, "buffer"):
    sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8', errors='replace')


# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_curve, auc
)

# Ensemble & Boosting Models
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Set visual style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Configuration
RANDOM_STATE = 42
TEST_SIZE = 0.2
FIGSIZE = (12, 8)

print("=" * 70)
print("NETWORK TRAFFIC CLASSIFICATION - AI CRAWLER TAR PITS RESEARCH")
print("=" * 70)
print(f"Execution Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("Libraries loaded successfully!")
print("=" * 70)

NETWORK TRAFFIC CLASSIFICATION - AI CRAWLER TAR PITS RESEARCH
Execution Time: 2026-02-04 15:04:06
Libraries loaded successfully!


In [9]:
# =========================
# BASIC DATASET INFORMATION
# =========================

print("\n" + "=" * 70)
print("DATASET INFORMATION")
print("=" * 70)

# Drop accidental index column if present
if 'Unnamed: 0' in df.columns:
    df.drop(columns=['Unnamed: 0'], inplace=True)

print(f"\n📊 Dataset Dimensions:")
print(f"   • Total Samples: {df.shape[0]:,}")
print(f"   • Total Features: {df.shape[1]}")

# Detect label column safely
label_col = 'Label' if 'Label' in df.columns else 'label'

# Defensive check (important for robustness)
if label_col not in df.columns:
    raise ValueError("Label column not found in dataset")

# Class distribution
attack_count = (df[label_col] == 1).sum()
normal_count = (df[label_col] == 0).sum()
total_count = len(df)

print(f"\n📈 Class Distribution:")
print(f"   • Attack Traffic (Label=1): {attack_count:,} "
      f"({attack_count / total_count * 100:.1f}%)")
print(f"   • Normal Traffic (Label=0): {normal_count:,} "
      f"({normal_count / total_count * 100:.1f}%)")

# Column names
print(f"\n📋 Column Names ({len(df.columns)} columns):")
print(df.columns.tolist())



DATASET INFORMATION

📊 Dataset Dimensions:
   • Total Samples: 92,212
   • Total Features: 58

📈 Class Distribution:
   • Attack Traffic (Label=1): 38,898 (42.2%)
   • Normal Traffic (Label=0): 53,314 (57.8%)

📋 Column Names (58 columns):
['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Bwd PSH Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'ACK Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg

In [8]:
# Basic dataset information
print("\n" + "=" * 70)
print("DATASET INFORMATION")
print("=" * 70)

print(f"\n📊 Dataset Dimensions:")
print(f"   • Total Samples: {len(df):,}")
print(f"   • Total Features: {df.shape[1]}")

# detecting column lable
label_col = 'Label' if 'Label' in df.columns else 'label'

print(f"\n📈 Class Distribution:")
print(f"   • Attack Traffic (Label=1): {len(df[df[label_col] == 1]):,} "
      f"({len(df[df[label_col] == 1]) / len(df) * 100:.1f}%)")
print(f"   • Normal Traffic (Label=0): {len(df[df[label_col] == 0]):,} "
      f"({len(df[df[label_col] == 0]) / len(df) * 100:.1f}%)")

print(f"\n📋 Column Names ({len(df.columns)} columns):")
print(df.columns)



DATASET INFORMATION

📊 Dataset Dimensions:
   • Total Samples: 92,212
   • Total Features: 59

📈 Class Distribution:
   • Attack Traffic (Label=1): 38,898 (42.2%)
   • Normal Traffic (Label=0): 53,314 (57.8%)

📋 Column Names (59 columns):
Index(['Unnamed: 0', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Bwd PSH Flags', 'Fwd Header Len', 'Bwd Header Len',
       'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max',
       'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt',
       '

In [10]:
print("\n" + "=" * 70)
print("EXPLORATORY DATA ANALYSIS (EDA)")
print("=" * 70)

# 3.1 statistical summary
print("\n📊 Statistical Summary of Numerical Features:")
print("-" * 50)
(df.describe())


EXPLORATORY DATA ANALYSIS (EDA)

📊 Statistical Summary of Numerical Features:
--------------------------------------------------


,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Act Data Pkts,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
count,9.221200e+04,92212.000000,92212.000000,9.221200e+04,9.221200e+04,92212.000000,92212.000000,92212.000000,92212.000000,92212.000000,...,92212.000000,9.221200e+04,9.221200e+04,9.221200e+04,9.221200e+04,9.221200e+04,9.221200e+04,9.221200e+04,9.221200e+04,92212.000000
mean,1.070781e+07,10.943370,12.974233,5.600684e+03,1.021176e+04,77.677287,6.762699,22.974344,25.156811,221.865007,...,5.779454,4.351079e+05,7.777843e+04,5.598308e+05,3.854448e+05,3.928015e+06,4.906831e+05,4.354137e+06,3.542860e+06,0.421832
std,2.708038e+07,249.027375,188.163610,2.857600e+05,2.096956e+05,227.595074,33.400015,76.539275,81.240567,501.244678,...,204.037943,1.516574e+06,7.175262e+05,1.983055e+06,1.416148e+06,1.200553e+07,3.847780e+06,1.334522e+07,1.143824e+07,0.493855
min,1.000000e+00,0.000000,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,2.182200e+04,0.000000,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
50%,7.014100e+04,1.000000,2.000000,0.000000e+00,1.160000e+02,0.000000,0.000000,0.000000,0.000000,70.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
75%,6.500053e+06,5.000000,3.000000,3.100000e+01,2.520000e+02,30.000000,0.000000,15.500000,0.000000,202.000000,...,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000
max,1.200000e+08,15123.000000,21933.000000,2.226106e+07,3.098852e+07,2920.000000,2442.000000,2442.000000,842.931393,18980.000000,...,15123.000000,1.082142e+08,5.927926e+07,1.082142e+08,1.082142e+08,1.199400e+08,7.695603e+07,1.199400e+08,1.199400e+08,1.000000
